<a href="https://colab.research.google.com/github/emilyj19/TS/blob/master/SQF_RNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install torch

In [0]:
import numpy as np 
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = 'gdrive/My Drive/FinancialTS/JPmarket_dataset.npz' 
data = np.load(root_path)
data.files

['train_ratios', 'test_ratios', 'train_volumes', 'test_volumes']

In [0]:
train_ratios = data['train_ratios']
test_ratios = data['test_ratios']
train_vols = data['train_volumes']
test_vols = data['test_volumes']

In [0]:
mini_train_ratios = train_ratios[0:50]
mini_test_ratios = test_ratios[0:50]

In [0]:
#need to change this to create windows for all stocks 

def split_data(n_days, data): #data for one stock in the form [days, bins]
  no_ts = data.shape[0]-n_days
  length_ts = n_days*data.shape[1]
  new_data = np.zeros((no_ts,length_ts))
  for j in range(no_ts):
    for i in range(n_days): 
      new_data[j,64*i:64*i + 64] = data[j+i,:]
  
  return new_data

training_data = split_data(180, train_ratios[0])

In [0]:
def make_test_data(train_data, test_data, n_days, stock_index):
  all_data = np.concatenate((train_data, test_data), axis = 1)
  
  one_data = all_data[stock_index]
  
  #now want to take only sets of n_days which contain the test data too 
  no_ts = test_data.shape[1]
  length_ts = n_days*one_data.shape[1]
  new_data = np.zeros((no_ts, length_ts))
  
  for j in range(no_ts): 
    for i in range(n_days): 
      new_data[j, 64*i:64*i + 64] = one_data[train_data.shape[1]+j-180+i+1,:]
      
  return new_data


testing_ratios = make_test_data(train_ratios, test_ratios, 180, 0)

In [0]:
class Encoder(nn.module): 
  
  def __init__(self, input_size, hidden_size, batch_size, output_size, num_layers):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.lstm = nn.LSTM(input_size = self.input_size, hidden_size = self.hidden_size, num_layers = self.num_layers)
    self.linear = nn.Linear(self.hidden_size, self.output_size)
    
  def init_hidden(self): 
    return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size),
              torch.zeros(self.num_layers, self.batch_size, self.hidden_size))
  
  def forward(self, data):
    lstm_out, self.hidden = self.lstm(data.view(len(data), self.batch_size, -1))
    
    return self.hidden #want the full sequence of hidden states as the output of the encoder layer

In [0]:
class Decoder(nn.module): 
  
  # want the decoder to share the same weights as the encoder - we won't train this?
  # but we need the decoder to take the output at the previous step to feed in as the next input 
  

In [0]:
class CRPS_loss(nn.module): 
  
  def __init__(self, ): 
    

In [0]:
class SQF_RNN(nn.module): 
  
  def __init__(self, encoder, decoder, loss_fn): 
    super(SQF_RNN, self).__init__()
    
    self.encoder = encoder
    self.decoder = decoder 
    self.loss_fn = loss_fn
   
    
  
  def forward(self, data): 
    output = self.encoder() #does this need to go within the class really?
  
    
    

In [0]:
learning_rate = 
learning_rate_decay = 